In [ ]:
############################# USING STUFF
using HDF5
using Plots
using Distributed
pyplot()

In [ ]:
############################# INPUT STUFF
input_name="24layered_011"

#filename=""

In [ ]:
############################# LOAD SAVE FILES
cd("..")
include("Initial_data/"*input_name*".jl")
file="./data/"*potname*"_R0($R0)_p0($p0).h5"
#file="MQC/data/"*filename*".h5"

In [ ]:
############################# HISTOGRAM STUFF
HISTO_RES=150;
xmin=-30;
xmax=32;

In [ ]:
###SO
SOfile="./data/SO_"*potname*"_R0($R0)_p0($p0).h5"
#SOfile="/home/nacho/Desktop/Desktop/HEAVY_DATA/SO_"*potname*"_R0($R0)_p0($p0).h5"


include("split_operator_functions_parallel.jl")
T_SO,X_SO,P_SO,PSI_AD_SO,PX_SO=SO_read(SOfile);           #LOAD SPLIT OPERATOR
Y,PX0=SO_histo_builder(X_SO,PX_SO[1],HISTO_RES,xmin,xmax);
Y,PXf=SO_histo_builder(X_SO,PX_SO[end],HISTO_RES,xmin,xmax);
PXf=PXf./maximum(PX0)
PX0=PX0./maximum(PX0);
###

In [ ]:
E0=h5read(file,"E0");                                     #LOAD INITIAL ENERGY

for dyn in DYN_LIST
    println("Loading $dyn...")
    if dyn in CL_LIST
        str="""T_$(dyn),R_$(dyn),P_$(dyn)=CL_read("$file","$dyn")"""
    elseif dyn in MF_LIST
        str="""T_$(dyn),R_$(dyn),P_$(dyn),C_$(dyn)=MF_read("$file","$dyn")"""
    elseif dyn in SH_LIST
        str="""T_$(dyn),R_$dyn,P_$(dyn),C_$(dyn),AST_$(dyn)=SH_read("$file","$dyn")"""
    end
    eval(Meta.parse(str))
end


for dyn in DYN_LIST
    str="Rbase_$dyn,HR_$dyn=super_histo(R_$dyn[:,1,:],$xmin,$xmax,$HISTO_RES)"
    eval(Meta.parse(str))
    str="HR_$dyn=HR_$dyn./maximum(HR_$dyn[:,1])"
    eval(Meta.parse(str))
end
println("Done!")

In [ ]:
N_AST=[sum(AST_CM2_FSSH[k,:])-120 for k in 1:101];

In [ ]:
CCM2_avg=[sum(abs2.(C_CM2_FSSH[k,2,:]))./120 for k in 1:101];

In [ ]:
plot(N_AST./120)
plot!(CCM2_avg)

In [ ]:
@manipulate for traj in 1:120
    plot(T_CM3_FSSH,abs2.(C_CM3_FSSH[:,1,traj]))
    plot!(T_CM3_FSSH,abs2.(C_CM3_FSSH[:,2,traj]))
    plot!(T_CM3_FSSH,abs2.(C_CM3_FSSH[:,3,traj]))
end

In [ ]:
############### HISTOGRAM PLOTTING: WITH SO
GUIDEFONT=font(24,"Helvetica");
TICKFONT=font(24,"Helvetica");
P=plot(Y,PX0,line=(2.0,:dash),color=:grey);
plot!(Y,PXf,line=(2.5,:dot),color=:black);
#P=plot(R_eh[:,1],HR_eh[:,1],label="EH_i",line=(1,:dash),color=:black);
plot!(Rbase_EH[:,end],-HR_EH[:,end],label="EH_f",line=(2,:solid),color=:blue);
plot!(Rbase_FSSH[:,end],HR_FSSH[:,end],label="FSSH_f",line=(2.5,:solid),color=:red);
#plot!(Rbase_FSSH_dia[:,end],HR_FSSH_dia[:,end],label="FSSH_d_f",line=(1.5,:solid),color=:green);
#plot!(Rbase_CM2[:,end],HR_CM2[:,end],label="CM2",line=(2,:solid))
#plot!(Rbase_CM3[:,end],HR_CM3[:,end],label="CM3",line=(2,:solid))
#plot!(Rbase_CM2_FSSH[:,end],HR_CM3_FSSH[:,end],label="CM2_F",line=(2,:solid))
#plot!(Rbase_CM3_FSSH[:,end],HR_CM3_FSSH[:,end],label="CM3_F",line=(2,:solid))
plot!(xlabel="Position (a.u.)",ylabel="Nuclear distribution",xlims=(xmin,xmax));
plot!(xguidefont = GUIDEFONT,xtickfont=TICKFONT,yguidefont = GUIDEFONT,ytickfont=TICKFONT);
#plot!(legend=false);
#ylims!(-0.25,0.25)

P


In [ ]:
############### HISTOGRAM PLOTTING: WITH SO
GUIDEFONT=font(24,"Helvetica");
TICKFONT=font(24,"Helvetica");
P=plot(Rbase_EH[:,1],HR_EH[:,1],label="EH_i",line=(1,:dash),color=:black);
plot!(Rbase_EH[:,end],-HR_EH[:,end],label="EH_f",line=(2,:solid),color=:blue);
plot!(Rbase_FSSH[:,end],HR_FSSH[:,end],label="FSSH_f",line=(2.5,:solid),color=:red);
#plot!(Rbase_FSSH_dia[:,end],HR_FSSH_dia[:,end],label="FSSH_d_f",line=(1.5,:solid),color=:green);
#plot!(Rbase_CM2[:,end],-HR_CM2[:,end],label="CM2",line=(2,:solid))
#plot!(Rbase_CM3[:,end],HR_CM3[:,end],label="CM3",line=(2,:solid))
plot!(Rbase_CM2_FSSH[:,end],-HR_CM2_FSSH[:,end],label="CM2_F",line=(2,:solid))
plot!(Rbase_CM3_FSSH[:,end],HR_CM3_FSSH[:,end],label="CM3_F",line=(2,:solid))
plot!(xlabel="Position (a.u.)",ylabel="Nuclear distribution",xlims=(xmin,xmax));
plot!(xguidefont = GUIDEFONT,xtickfont=TICKFONT,yguidefont = GUIDEFONT,ytickfont=TICKFONT);
#plot!(legend=false);
#ylims!(-0.25,0.25)

P
